<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/BaseAut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from openpyxl import load_workbook

# Ruta base al directorio donde están los archivos
directorio = os.path.join(os.path.dirname(__file__), "FU")

# Filtra los archivos de Excel
archivos = [
    f for f in os.listdir(directorio)
    if f.endswith((".xlsx", ".xls", ".XLSX", ".XLS")) and not f.startswith("~$")
]

dataframes = {}
importaciones = {}
nombre_hoja = "Aéreos"

for archivo in archivos:
    ruta_archivo = os.path.join(directorio, archivo)
    nombre = os.path.splitext(archivo)[0]

    try:
        # Cargar el archivo completo con pandas
        df = pd.read_excel(ruta_archivo)
        dataframes[nombre] = df

        # Si es el archivo de Control de Importaciones, procesar hoja "Aéreos"
        if "CONTROL DE IMPORTACIONES" in archivo:
            try:
                wb = load_workbook(ruta_archivo, data_only=True)
                if nombre_hoja in wb.sheetnames:
                    ws = wb[nombre_hoja]
                    data = [row for row in ws.iter_rows(values_only=True)]

                    headers = data[0]
                    rows = data[1:]

                    df_aereos = pd.DataFrame(rows, columns=headers)
                    importaciones["CONTROL DE IMPORTACIONES"] = df_aereos

                    print("Hoja 'Aéreos' cargada exitosamente.")
                else:
                    print(f"La hoja '{nombre_hoja}' no existe en {ruta_archivo}.")
            except Exception as e:
                print(f"Error al procesar '{ruta_archivo}': {e}")

    except Exception as e:
        print(f"Error al leer {archivo}: {e}")

# Muestra cuáles cargaste
print(f"Archivos cargados:", list(dataframes.keys()))

df_base = "BASE"
if df_base in dataframes:
    df_base = dataframes[df_base]
else:
    print(f"No se encontró el archivo {df_base} en los archivos cargados")

df_base = df_base.iloc[1:].reset_index(drop=True)

columnas_a_eliminar = [
    "Seq. No. of Account Assgt",
    "Cost Center","WBS Element",
    "Order", 'Purchasing Group',
    "Asset","Sub-number",
    "Network","Activity",
    "Purchasing Doc. Type",
    "Purch. doc. category",
    'PO history/release documentation',
    'Deletion Indicator','Item Category',
    'Acct Assignment Cat.',
    'Acc. assgt quantity',
    'Price unit', 'No. of Positions'
    ]

for col in columnas_a_eliminar:
    if col not in df_base.columns:
        print(f" La columna '{col}' no está en el df y será ignorada.")
df_base_limpio = df_base.drop(columns=[col for col in columnas_a_eliminar if col in df_base.columns])

columnas_renombrar = {
    'Purchasing Document' : 'PO',
    'Item' : 'Posición PO',
    'SD Document': 'SO KMEX',
    'Item.1' : 'Posición KMEX',
    'Document Date' : 'Fecha de PO',
    'Vendor/supplying plant' : 'Proveedor/Centro suministrador',
    'Short Text' : 'Texto breve',
    'Material Group' : 'Grupo de articulos',
    'Plant': 'Centro',
    'Storage Location': 'Almacén',
    'Order Quantity': 'Cantidad de Pedido',
    'Order Unit': 'Unidad medida pedido',
    'Net price': 'Precio neto',
    'Currency': 'Moneda'
}

df_base_limpio = df_base_limpio.rename(columns={k: v for k, v in columnas_renombrar.items() if k in df_base_limpio.columns})
print("Columnas despues de ajustar las especificaciones:")
print(df_base_limpio.columns.tolist())

#=IFERROR(VLOOKUP([@[SO KMEX]],[VA05 KMEX.xlsx]Sheet1'!$A:$C,3,0),"")

if "VA05 KMEX" in dataframes:
    df_ref = dataframes["VA05 KMEX"]
    df_base_limpio['SO KMEX'] = df_base_limpio['SO KMEX'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_ref['Document Date'] = df_ref['Document Date'].astype(str).str.strip()
    df_ref['SD Document'] = df_ref['SD Document'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    df_ref = df_ref.drop_duplicates(subset = ['SD Document'])

    # Simulación de VLOOKUP usando merge
    df_base_limpio = df_base_limpio.merge(
        df_ref[['SD Document', 'Document Date']],
        how='left',  # como VLOOKUP
        left_on = 'SO KMEX',
        right_on='SD Document'
    )

    # Simulación de IFERROR (rellena valores faltantes)
    df_base_limpio['Fecha SO KMEX'] = df_base_limpio['Document Date'].fillna('')

    df_base_limpio = df_base_limpio.drop(columns=['SD Document', 'Document Date'])

    print("Primer VLOOKUP")

    print("Coincidencias encontradas: ", (df_base_limpio['Fecha SO KMEX'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['Fecha SO KMEX'] == '').sum())
    print(len(df_base_limpio))
else:
    print("No se encontró el archivo EXW KAG en los archivos cargados. No se puede traer EXW Real.")


#=IFERROR(VLOOKUP([@PO],[SO KAG.XLSX]Sheet1'!$B:$E,4,0),"")

# Si existe el archivo SO KAG y las columnas necesarias
if "SO KAG" in dataframes:
    df_so_kag = dataframes["SO KAG"]
    # Asegura que ambas claves sean string y limpias
    df_base_limpio['PO'] = df_base_limpio['PO'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_so_kag['SD Document'] = df_so_kag['SD Document'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_so_kag['Purchase Order Number'] = df_so_kag['Purchase Order Number'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    df_so_kag = df_so_kag.drop_duplicates(subset = ['Purchase Order Number', 'SD Document'])
    df_so_kag = df_so_kag.groupby('Purchase Order Number')['SD Document'].first().reset_index()

    # Realiza el merge para traer 'Documento Comercial' desde SO KAG
    df_base_limpio = df_base_limpio.merge(
        df_so_kag[['Purchase Order Number', 'SD Document']],
        how='left',
        left_on='PO',
        right_on='Purchase Order Number'
    )

    df_base_limpio['SO KAG'] = df_base_limpio['SD Document'].fillna('')
    df_base_limpio = df_base_limpio.drop(columns=['Purchase Order Number', 'SD Document'])
    print("Segundo VLOOKUP")
    print("Coincidencias encontradas: ", (df_base_limpio['SO KAG'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['SO KAG'] == '').sum())
else:
    print("No se encontró el archivo SO KAG en los archivos cargados. No se puede traer Delivery Note.")

print("Tercer VLOOKUP")
# Agregar columna 'Key' vacía.
df_base_limpio['Key SO'] = ""

df_base_limpio['Key SO'] = df_base_limpio['Key SO'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
df_base_limpio['Material'] = df_base_limpio['Material'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
df_base_limpio['SO KAG'] = df_base_limpio['SO KAG'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

# Operación & en excel.
df_base_limpio['Key SO'] = df_base_limpio['SO KAG'].astype(str).str.strip() + df_base_limpio['Material'].astype(str).str.strip()

#=IFERROR(VLOOKUP([@Key],[EXW KAG.XLSX]Sheet1'!$A:$F,6,0),"")

# Si existe el archivo EXW KAG y las columnas necesarias
if "EXW KAG" in dataframes:
    df_exw_kag = dataframes["EXW KAG"]
    # Asegura que ambas claves sean string y limpias
    df_base_limpio['Key SO'] = df_base_limpio['Key SO'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_exw_kag['Material'] = df_exw_kag['Material'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_exw_kag['Document'] = df_exw_kag['Document'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_exw_kag['Key EXW'] = df_exw_kag['Document'] + df_exw_kag['Material']
    df_exw_kag['Key EXW'] = df_exw_kag['Key EXW'].astype(str).str.strip()
    df_exw_kag['Entrega'] = df_exw_kag['Entrega'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    df_exw_kag = df_exw_kag.drop_duplicates(subset = ['Key EXW'])

    # Realiza el merge para traer 'Entrega' desde EXW KAG
    df_base_limpio = df_base_limpio.merge(
        df_exw_kag[['Key EXW', 'Entrega']],
        how='left',
        left_on='Key SO',
        right_on='Key EXW'
    )
    df_base_limpio['Delivery Note'] = df_base_limpio['Entrega'].fillna('')
    df_base_limpio = df_base_limpio.drop(columns=['Key EXW', 'Entrega'])
    print("Cuarto VLOOKUP")
    print("Coincidencias encontradas: ", (df_base_limpio['Delivery Note'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['Delivery Note'] == '').sum())
    print(len(df_base_limpio))
else:
    print("No se encontró el archivo EXW KAG en los archivos cargados. No se puede traer Delivery Note.")

#=IFERROR(VLOOKUP([@Key],[SO KAG.XLSX]Sheet1'!$A:$M,13,0),"")
if "SO KAG" in dataframes:
    df_so_kag = dataframes["SO KAG"]

    # En df_base_limpio
    df_base_limpio['PO'] = df_base_limpio['PO'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_base_limpio['Key SO'] = df_base_limpio['Key SO'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    # En df_so_kag
    df_so_kag['Material'] = df_so_kag['Material'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_so_kag['SD Document'] = df_so_kag['SD Document'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_so_kag['Key S'] = df_so_kag['SD Document'] + df_so_kag['Material']
    df_so_kag['Key S'] = df_so_kag['Key S'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_so_kag['Delivery Date'] = df_so_kag['Delivery Date'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    df_so_kag = df_so_kag.drop_duplicates(subset = ['Key S'])

    # Hacemos el merge para traer 'Fecha de entrega' como EXW Estimado
    df_base_limpio = df_base_limpio.merge(
        df_so_kag[['Key S', 'Delivery Date']],
        how='left',
        left_on='Key SO',
        right_on='Key S'
    )

    # Creamos la columna final con el nombre solicitado
    df_base_limpio['EXW Estimado'] = df_base_limpio['Delivery Date'].fillna('')

    df_base_limpio = df_base_limpio.drop(columns=['Key S', 'Delivery Date'])

    print("Quinto VLOOKUP")
    print("Coincidencias encontradas: ", (df_base_limpio['EXW Estimado'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['EXW Estimado'] == '').sum())
    print(len(df_base_limpio))
else:
    print("No se encontró el archivo SO KAG en los archivos cargados. No se puede traer EXW Estimado.")

if "EXW KAG" in dataframes:
    df_exw_kag = dataframes["EXW KAG"]
    # Asegura que ambas claves sean string y limpias
    df_base_limpio['Key SO'] = df_base_limpio['Key SO'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_exw_kag['Key EXW'] = df_exw_kag['Key EXW'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_exw_kag['Conf. list price'] = df_exw_kag['Conf. list price'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    df_exw_kag = df_exw_kag.drop_duplicates(subset = ['Key EXW'])

    # Realiza el merge para traer 'Entrega' desde EXW KAG
    df_base_limpio = df_base_limpio.merge(
        df_exw_kag[['Key EXW', 'Conf. list price']],
        how='left',
        left_on='Key SO',
        right_on='Key EXW'
    )
    df_base_limpio['EXW Real'] = df_base_limpio['Conf. list price'].fillna('')
    df_base_limpio = df_base_limpio.drop(columns=['Key EXW', 'Conf. list price'])
    print("Sexto VLOOKUP")
    print("Coincidencias encontradas: ", (df_base_limpio['EXW Real'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['EXW Real'] == '').sum())
    print(len(df_base_limpio))
else:
    print("No se encontró el archivo EXW KAG en los archivos cargados. No se puede traer EXW Real.")

#=IFERROR(VLOOKUP([@Key],[EXW KAG.XLSX]Sheet1'!$A:$L,12,0),"")

if "EXW KAG" in dataframes:
    df_exw_kag = dataframes["EXW KAG"]
    # Asegura que ambas claves sean string y limpias
    df_base_limpio['Key SO'] = df_base_limpio['Key SO'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_exw_kag['Key EXW'] = df_exw_kag['Key EXW'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_exw_kag['Cantidad entrega'] = df_exw_kag['Cantidad entrega'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    df_exw_kag = df_exw_kag.drop_duplicates(subset = ['Key EXW'])

    # Realiza el merge para traer 'Entrega' desde EXW KAG
    df_base_limpio = df_base_limpio.merge(
        df_exw_kag[['Key EXW', 'Cantidad entrega']],
        how='left',
        left_on='Key SO',
        right_on='Key EXW'
    )
    df_base_limpio['EXW Real'] = df_base_limpio['Cantidad entrega'].fillna('')
    df_base_limpio = df_base_limpio.drop(columns=['Key EXW', 'Cantidad entrega'])
    print("Septimo VLOOKUP")
    print("Coincidencias encontradas: ", (df_base_limpio['EXW Real'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['EXW Real'] == '').sum())
    print(len(df_base_limpio))
else:
    print("No se encontró el archivo EXW KAG en los archivos cargados. No se puede traer Cantidad entrega.")

if "CONTROL DE IMPORTACIONES" in dataframes:
    df_import = dataframes["CONTROL DE IMPORTACIONES"]

    # Limpia y asegura tipo string de las claves de ambos lados
    df_base_limpio['Delivery Note'] = df_base_limpio['Delivery Note'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_import['DN'] = df_import['DN'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    print(df_base_limpio['Delivery Note'].drop_duplicates().head())
    print(df_import['DN'].drop_duplicates().head())

    cols_a_traer = ['DN', 'Número de consol', 'Pedimento', 'Fecha entrada a aduana', 'Fecha pago pedimento']

    df_import_f = df_import[cols_a_traer].drop_duplicates(subset=['DN'])

    df_base_limpio = df_base_limpio.merge(
        df_import_f,
        how='left',
        left_on='Delivery Note',
        right_on='DN'
    )

    for col in ['Número de consol', 'Pedimento', 'Fecha entrada a aduana']:
        if col in df_base_limpio.columns:
            df_base_limpio[col] = df_base_limpio[col].fillna('')

    df_base_limpio['Fecha de pago pedimento'] = df_base_limpio['Fecha pago pedimento'].fillna('')
    df_base_limpio = df_base_limpio.drop(columns = ['Fecha pago pedimento'])

    df_base_limpio = df_base_limpio.drop(columns=['DN'])
    print("Octavo VLOOKUP")
    print("Coincidencias encontradas: ", (df_base_limpio['Número de consol'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['Número de consol'] == '').sum())

    print("Coincidencias encontradas: ", (df_base_limpio['Pedimento'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['Pedimento'] == '').sum())

    print("Coincidencias encontradas: ", (df_base_limpio['Fecha entrada a aduana'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['Fecha entrada a aduana'] == '').sum())

    print("Coincidencias encontradas: ", (df_base_limpio['Fecha de pago pedimento'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['Fecha de pago pedimento'] == '').sum())
    print(len(df_base_limpio))

else:
    print("No se encontró el archivo CONTROL DE IMPORTACIONES en los archivos cargados. No se pueden traer las columnas solicitadas.")

if "CONTROL DE IMPORTACIONES" in importaciones:

    df_aereos = importaciones["CONTROL DE IMPORTACIONES"]

    # Limpia los campos clave
    df_base_limpio['Delivery Note'] = df_base_limpio['Delivery Note'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_aereos['DN'] = df_aereos['DN'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    cols_a_traer = ['DN', 'Guía', 'Fecha pago pedimento']

    df_aereos_f = df_aereos[cols_a_traer].drop_duplicates(subset=['DN'])

    df_base_limpio = df_base_limpio.merge(
        df_aereos_f,
        how='left',
        left_on='Delivery Note',
        right_on='DN'
    )

    df_base_limpio['Aéreos'] = df_base_limpio['Guía'].fillna('')
    df_base_limpio['Fecha pago pedimento aereo'] = df_base_limpio['Fecha pago pedimento'].fillna('')

    print("Noveno VLOOKUP")

    print("Coincidencias encontradas: ", (df_base_limpio['Aéreos'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['Aéreos'] == '').sum())

    print("Coincidencias encontradas: ", (df_base_limpio['Fecha pago pedimento aereo'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['Fecha pago pedimento aereo'] == '').sum())

    print(len(df_base_limpio))

else:
    print("No se encontró la hoja 'Aéreos' en los archivos cargados.")

if "101 EM" in dataframes:
    df_101em = dataframes["101 EM"]

    # --- Armar el pivote Key101 en ambos DataFrames--
    df_101em['Sales Order'] = df_101em['Sales Order'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_101em['Sales order item'] = df_101em['Sales order item'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_101em['Key101'] = df_101em['Sales Order'] + df_101em['Sales order item']
    df_101em['Key101'] = df_101em['Key101'].str.replace(r'\.0$', '', regex = True).astype(str).str.strip()

    df_base_limpio['SO KMEX'] = df_base_limpio['SO KMEX'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()
    df_base_limpio['Posición KMEX'] = df_base_limpio['Posición KMEX'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    #Creación de Key2
    df_base_limpio['Key2'] = df_base_limpio['SO KMEX'] + df_base_limpio['Posición KMEX']
    df_base_limpio['Key2'] = df_base_limpio['Key2'].astype(str).str.replace(r'\.0$', '', regex = True).str.strip()

    # --- Traer ambas columnas en un solo merge ---
    cols_traer = ['Key101', 'Entry Date', 'Quantity']
    df_101em_filtrado = df_101em[cols_traer].drop_duplicates(subset=['Key101'])
    df_base_limpio = df_base_limpio.merge(
        df_101em_filtrado,
        how='left',
        left_on='Key2',
        right_on='Key101'
    )
    df_base_limpio['Fecha EM'] = df_base_limpio['Entry Date'].fillna('')
    df_base_limpio['Cantidad EM'] = df_base_limpio['Quantity'].fillna('')
    df_base_limpio = df_base_limpio.drop(columns=['Entry Date', 'Quantity', 'Key101'])

    print("Decimo VLOOKUP")

    print("Coincidencias encontradas: ", (df_base_limpio['Fecha EM'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['Fecha EM'] == '').sum())
    print("Coincidencias encontradas: ", (df_base_limpio['Cantidad EM'] != '').sum())
    print("Sin coincidencias encontradas: ", (df_base_limpio['Cantidad EM'] == '').sum())
    print(len(df_base_limpio))
else:
    print("No se encontró el archivo 101EM en los archivos cargados.")

print(df_base_limpio.head())

#DEJAR ESTA LINEA AL FINAL PARA SOBREESCRIBIR LOS CAMBIOS; TENER EL DOCUMENTO CERRADO
#ruta_guardado = os.path.join(directorio, "Follow-up SPS DC03(PY).xlsx")
#df_base_limpio = df_base_limpio.fillna('')
#df_base_limpio.to_excel(ruta_guardado, index = False)
#print("Archivo guardado en: ", ruta_guardado)




